# Direct Reduction of Iron

In [1]:
# Dependencies.
from IPython.display import HTML, Markdown

import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"

from posted import TEDF


# Set variable of TEDF.
var = "Tech|Direct Reduction of Iron"

# Loading the TEDF.
tedf = TEDF.load(var)

# Define units to use for energy carriers.
units={
    "Electricity": "MWh",
    "Heat": "MWh",
    "Natural Gas": "MWh_NG_LHV",
    "Hydrogen": "MWh_H2_LHV",
}

## Fields

The techno-economic data is distinguished across the following additional fields.

### Operation mode (`mode`)

In [2]:
Markdown(
    "\n".join(f"* **{code}**: {desc}" for code, desc in tedf.fields["mode"].codes.items())
)

* **H2**: Operate fully with hydrogen.
* **NG**: Operate fully with natural gas.

## Aggregated parameters

All data added to the POSTED database is aggregated automatically using the POSTED framework. The result yields the following parameters:

In [3]:
aggregated = tedf.aggregate(units=units, append_references=True)

display(
    aggregated
    .pivot(
        index=aggregated.columns[:-3],
        columns=["variable", "unit"],
        values="value",
    )
    .map(lambda x: float(f"{x:.3g}") if not pd.isnull(x) else x)
    .fillna("")
)

Loading ITables v2.6.2 from the internet... (need help?)


## Energy demand

The figure below gives an overview of the energy demand reported by different sources across different energy carriers.

In [4]:
selected = tedf.select(units=units)
aggregated = tedf.aggregate(units=units)

df_plot = (
    pd.concat([
        selected,
        aggregated.assign(source="POSTED-default"),
    ])
    .assign(variable=lambda df: df["variable"].str.extract(r"^Input\|(.*)", expand=False))
    .query(f"variable.isin({list(units)})")
    .assign(variable=lambda df: df.agg(lambda r: f"{r['variable']}{(' (' + r['component'] + ')') if r['component'] not in ['#', np.nan] else ''}", axis=1))
)

display(
    df_plot
    .plot.bar(
        x="source",
        y="value",
        color="variable",
        facet_col="mode",
    )
    .update_xaxes(
        title=None,
    )
    .for_each_annotation(
        lambda a: a.update(text=": ".join(a.text.split("=")))
    )
    .update_layout(
        legend_title=None,
        xaxis_title=None,
        yaxis_title="Energy demand per {reference_variable}  [ MWh / {reference_unit} ]".format(**df_plot.iloc[0]),
    )
)

## Raw data

In [5]:
Markdown(f"""
The table below contains the raw data contained in the public POSTED database. This data has not be automatically normalised or harmonised in any way. You can also find this data in the GitHub repo in this file:
{link_public_github(var)}
""")


The table below contains the raw data contained in the public POSTED database. This data has not be automatically normalised or harmonised in any way. You can also find this data in the GitHub repo in this file:
<a href="https://github.com/PhilippVerpoort/posted/blob/main/posted/database/tedfs/Tech/Direct Reduction of Iron.csv">posted/database/tedfs/Tech/Direct Reduction of Iron.csv</a>


In [6]:
edit(tedf)

Loading ITables v2.6.2 from the internet... (need help?)
